http://cl-informatik.uibk.ac.at/isr24/ rewriting school. Seems like all the good topics
https://link.springer.com/book/10.1007/978-3-031-43369-6 frontiers of combining systems

polynomial interpretations
matrix interpretations
SAT/LP solver for termination
dealing wiht the xml format
How does the frontend deal with things

There is rewriting modulo equations, realtive rewriting, innermost rewriting, conditional rewriting, outermost rewriting, higher order rewriting hrs, cycle rewriting, string rewriting. How are these dealt with?

Confluence checking CSI
https://en.wikipedia.org/wiki/Cycle_detection

Java. Hmm.
eBPF

https://maude.cs.uiuc.edu/tools/mta/
https://maude.cs.uiuc.edu/tools/itp/hendrix-thesis.pdf
hendrix thesis. Equational tree automata

Well founded induction

if foo(s(x)) -> foo(x) is a well founded definition, then there is intrinisically a valid induction hypothesis in there
Is extracting the piece related the dependency pairs?
How to deal with completeness check? Default to some dummy value? Well, it isn't a problem. One can't prove what the value is on other pieces. We do want confluence though. To make sure we don't have foo(x) == true /\ foo(x) == false

higher order. sizechangetool  https://github.com/Deducteam/SizeChangeTool wanda 

## Dependency Pairs
https://www.sciencedirect.com/science/article/pii/S0304397599002078 Termination of term rewriting using dependency pairs
Dependency pairs  fraemwork http://cl-informatik.uibk.ac.at/users/thiemann/paper/LPAR04DPFramework.pdf

https://drops.dagstuhl.de/storage/00lipics/lipics-vol131-fscd2019/LIPIcs.FSCD.2019.9/LIPIcs.FSCD.2019.9.pdf Dependency Pairs Termination in Dependent Type Theory Modulo Rewriting

Wanda 

"Dependency pairs aren't that mysterious:
Plus(x, S(y)) -> S(Plus(x, y))
gives the pair
Plus(x, S(y)) -> Plus(x, y)
which intuitively is the only bit we really care about re termination.
This is the "ignore the context" bit you were mentioning.
"

"
Dependency pairs: not that tough an idea
take a rewrite system R = { l -> r }
Define G to be like R, but with every possible subterm of each right hand side
Dis association — Yesterday at 10:11 PM
so if G is f(x, y) -> g(f(y, x), y) then G is f(x, y) -> g(f(y, x), y) and f(x, y) -> f(y, x)
now only allow G steps at the root, and then some arbitrary number of R steps not at the root
the theorem is that termination of R <=> there is no infinite reduction of G-root + R non-root steps
the idea is that G "focuses" the reductions on each possible subterm, ignoring the context
now you only need to care about possible infinite sequences of G steps (with in-between R steps)
philzook — Yesterday at 10:13 PM
Seems worse
G is bigger than r
Dis association — Yesterday at 10:14 PM
yes but
you can observe that some G steps may not follow another G step
e.g.
times(x, S(y)) -> plus(x, times(x, y))
the G steps are times(x, S(y)) -> plus(x, times(x, y)) and times(x, S(y)) -> times(x, y)
but there is no outer times rule for times after the first step!
so the only "loop" for times is the second one
the idea is that you can treat each "mutual recursion" rules individually
often to make this explicit they rename the G head symbols
so G is actually TIMES(x, S(y)) -> PLUS(x, times(x, y)) and TIMES(x, S(y)) -> TIMES(x, y)
which feels different
there's  only one rules that can lead to infinite TIMES steps
philzook — Yesterday at 10:19 PM
But we still have all of R
Dis association — Yesterday at 10:19 PM
but what you need to show is termination of alternating G/R steps
so if you prove that there can't be infinitely many G steps you're golden
you'll see that in AproVE that's what they're trying to do
"remove" G rules
in practice you replace "non-trivial" inner terms by some fresh var 
so TIMES(x, S(y)) -> PLUS(x, z)
that means any plus rule can fire at this point
but a PLUS rule can't reach a TIMES rule, so you can prove first termination of PLUS then termination of TIMES
but read the paper it's well-written
philzook — Yesterday at 10:29 PM
Plus doesnt depend on times. Is that not sufficient to see they can be treated hierarchically?
I cant show plus terminates then leave its rules out and show times terminates?
How could termination checking possibly work in coq then
Dis association — Yesterday at 10:51 PM
Yes you can, but in TRSes things are not generally so modular
"depends on" is basically what the dependency pair relation captures
you can have crazy rules like plus(x, x) -> times(2, x)
does that terminate?
anyways
philzook — Today at 8:12 AM
I’m suspicious."

simplification pair 
reduction order
order eakly, delete strict pieces
SCC dependency graph


https://aprove.informatik.rwth-aachen.de/download

https://matryoshka-project.github.io/wait2018/ WAIT Workshop on Automated (Co)inductive Theorem Proving
HipSpec https://www.cse.chalmers.se/~jomoa/papers/hipspec-atx.pdf
https://dmg.tuwien.ac.at/hetzl/teaching/d_vierling.pdf The limits of automated
inductive theorem provers 2023 thesis
vampire has induction support. What does it look like?
https://www.cse.chalmers.se/~jomoa/papers/cicm15-TIP.pdf tons of inductive problems https://github.com/tip-org


In [5]:
%%file /tmp/prolog.pl
%query: append(b,f,f)
append(nil,L,L).
append(cons(X,Xs),Ys,cons(X, Zs)) :- append(Xs,Ys,Zs).

Overwriting /tmp/prolog.pl


In [3]:
! java -ea -cp ~/Documents/solvers/aprove.jar aprove.CommandLineInterface.PrologFrontendMain 

No input file, aborting.
Usage: java -cp aprove.jar aprove.CommandLineInterface.PrologFrontendMain [OPTION] PLFILE

Prolog to QDP/Graph dump export from AProVE 2014.

 -h, --help             print this help
 -o, --outputDir DIR    directory in which TRSs will be dumped (default '.')
 -t, --timeout SECONDS  timeout, in seconds (default 60)
 -p, --proof            print proof for steps from input to TRSs
 -g, --graph yes|no     export to Graph (default no)
 -q, --query QUERY      a query which tells AProVE what to analyze
 -s, --startTerm TERM   analyze termination starting with term TERM
 -j, --json yes|no      export JSON (default no), conflicts with other output


In [6]:
! java -ea -cp ~/Documents/solvers/aprove.jar aprove.CommandLineInterface.PrologFrontendMain /tmp/prolog.pl

Dumped to ./prolog.pl-obl-4.qdp


In [ ]:
(VAR T17 T18)
(PAIRS
  F1_IN(cons(T17, T18)) -> U1^1(f1_in(T18), cons(T17, T18))
  F1_IN(cons(T17, T18)) -> F1_IN(T18)
)
(RULES
  f1_in(nil) -> f1_out1
  f1_in(cons(T17, T18)) -> U1(f1_in(T18), cons(T17, T18))
  U1(f1_out1, cons(T17, T18)) -> f1_out1
)
(Q
)
(EDGES
    2 -> 1
    2 -> 2
)
(MINIMAL)

In [7]:
%%file /tmp/hask.hs
{-# htermination (foldr1 :: (a -> a -> a) -> (List a) -> a) #-}


import qualified Prelude


data MyBool = MyTrue | MyFalse

data List a = Cons a (List a) | Nil


foldr1 :: (a -> a -> a) -> (List a) -> a

foldr1 f (Cons x Nil) = x

foldr1 f (Cons x xs) = f x (foldr1 f xs)

Overwriting /tmp/hask.hs


In [9]:
! java -ea -cp ~/Documents/solvers/aprove.jar aprove.CommandLineInterface.HaskellFrontendMain  /tmp/hask.hs

Dumped to ./hask.hs-obl-14.qdp
Dumped to ./hask.hs-obl-16.qdp


In [ ]:
(VAR vx3 vx40 vx410 vx411 h)
(PAIRS
  new_foldr1(vx3, Cons(vx40, Cons(vx410, vx411)), h) -> new_foldr1(vx3, Cons(vx410, vx411), h)
)
(RULES
)
(Q
)
(EDGES
    1 -> 1
)
(MINIMAL)